# Исследование надежности заемщиков.

Целью данного проекта является определение степени влияния различных факторов (в частности, семейное положение и количество детей клиента) на погашение кредита в срок.

Исходные данные банка — статистика о платёжеспособности клиентов. Описание данных приведено ниже.

### Шаг 1. Обзор данных

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')

In [2]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблице 12 столбцов:
* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

Число строк в столбцах `days_employed` и `total_income` не соответсвует общему числу строк, значит, в этих столбцах присутствуют пропуски.

### Шаг 2.1 Заполнение пропусков

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [5]:
data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В столбцах `days_employed` и `total_income` присутствует 2174 пропущенных значения, что составляет 10,1% от общего объема строк. 

In [6]:
data[(data['days_employed'].isna() == True) | (data['total_income'].isna() == True)].head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Вероятно, пропуски возникли из-за того, что на момент подачи кредитной заявки люди не предоставили документы, подтверждающие трудовой стаж и ежемесячный доход.

`days_employed` и `total_income` - количественные переменные, в которых некоторые значения могут сильно выделяться среди большинства, поэтому имеет смысл заполнить пропуски медианным значением. В противном случае придется отказаться от существенной доли выборки.

Прежде чем заполнять пропуски медианным значением следует проверить данные на аномалии, поскольку они могут привести к неправильному расчёту медианы.

### Шаг 2.2 Проверка данных на аномалии и исправления.

In [7]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Из представленной таблицы видно, что в столбцах `children` и `days_employed` присутствуют отрицательные значения. Вероятно, они связаны с человеческим фактором. 

In [8]:
data['children'] = data['children'].abs()
data['days_employed'] = data['days_employed'].abs()
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.543275,66914.728907,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.379876,139030.880527,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,0.000000,24.141633,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,927.009265,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,2194.220567,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,5537.882441,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Ещё в таблице видно, что среднее значение общего трудового стажа в днях составляет 60277 дней или 165 лет. Посмотрим, сколько строк имеют значение стажа больше, чем средний возраст клиента (42 года).

In [9]:
data[data['days_employed'] > 42 * 365]['days_employed'].count()

3455

3455 строк или 16,04% данных имеют аномальные значения в столбце `days_employed`. Удалить их нельзя, поскольку они составляют существенную долю выборки. Скорее всего, они возникли из-за технической ошибки, либо человеческого фактора.

Заменим аномальные значения на средние значения стажа для людей схожего возраста.

In [10]:
def correct_days_employed(row):
    row_number = row.name
    days_employed = row['days_employed']
    dob_years = row['dob_years']
    target = 42 * 365
    
    if days_employed > target:
        data.loc[row_number, 'days_employed'] = data[(data['days_employed'] <= target) & (data['dob_years'] == dob_years)]['days_employed'].mean()

data.apply(correct_days_employed, axis=1)
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.543275,2552.346135,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.379876,2124.640789,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,0.000000,24.141633,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,926.925347,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,2186.168272,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,3577.539706,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,15267.541183,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Теперь максимальный трудовой стаж составляет 15267 дней или 41 год. Однако также можно заметить, что в столбце `children` максимальное значение составляет 20, что сомнительно в современных реалиях.

In [11]:
data[data['children'] == 20]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,880.221113,21,среднее,1,женат / замужем,0,M,компаньон,0,145334.865002,покупка жилья
720,20,855.595512,44,среднее,1,женат / замужем,0,F,компаньон,0,112998.738649,покупка недвижимости
1074,20,3310.411598,56,среднее,1,женат / замужем,0,F,сотрудник,1,229518.537004,получение образования
2510,20,2714.161249,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474.835577,операции с коммерческой недвижимостью
2941,20,2161.591519,0,среднее,1,женат / замужем,0,F,сотрудник,0,199739.941398,на покупку автомобиля
...,...,...,...,...,...,...,...,...,...,...,...,...
21008,20,1240.257910,40,среднее,1,женат / замужем,0,F,сотрудник,1,133524.010303,свой автомобиль
21325,20,601.174883,37,среднее,1,женат / замужем,0,F,компаньон,0,102986.065978,профильное образование
21390,20,NaN,53,среднее,1,женат / замужем,0,M,компаньон,0,NaN,покупка жилой недвижимости
21404,20,494.788448,52,среднее,1,женат / замужем,0,M,компаньон,0,156629.683642,операции со своей недвижимостью


Учитывая, что существует 76 клиентов с 20 детьми, причем некоторым клиентам 20-30 лет, можно полноценно считать, что это аномалия. Скорее всего, она возникла из-за технической ошибки, либо человеческого фактора.

Достоверно восстановить данные не представляется возможным, поскольку, судя по характеру ошибки, у данных клиентов может быть либо двое детей, либо более 5. Также 76 строк составляют незначительный процент от выборки, поэтому данные значения можно удалить.

In [12]:
data = data[data['children'] <= 5]
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21449.000000,19284.000000,21449.000000,21449.000000,21449.000000,21449.000000,1.928400e+04
mean,0.474334,2553.309069,43.298615,0.817148,0.973099,0.080796,1.674178e+05
std,0.751417,2124.219168,12.576314,0.548507,1.420521,0.272528,1.030418e+05
min,0.000000,24.141633,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,927.913282,33.000000,1.000000,0.000000,0.000000,1.030161e+05
50%,0.000000,2188.801715,42.000000,1.000000,0.000000,0.000000,1.450208e+05
75%,1.000000,3577.539706,53.000000,1.000000,1.000000,0.000000,2.033700e+05
max,5.000000,15267.541183,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Также можно заметить, что в столбце `dob_years` присутствуют нулевые значения, т.е. возраст клиента равен 0, что является аномалией. Вероятно, эти значения появились из-за технической ошибки, либо человеческого фактора.

In [13]:
data[data['dob_years'] == 0]['dob_years'].count()

100

Строк с такими значениями 100 штук. К сожалению, восстановить такие данные не представляется возможным, поэтому их тоже придётся удалить из выборки.

In [14]:
data = data[data['dob_years'] > 0]
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21349.000000,19194.000000,21349.000000,21349.000000,21349.000000,21349.000000,1.919400e+04
mean,0.474308,2554.961940,43.501429,0.817837,0.971802,0.080800,1.674625e+05
std,0.751214,2124.792344,12.250774,0.548584,1.419916,0.272534,1.031558e+05
min,0.000000,24.141633,19.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,927.654762,33.000000,1.000000,0.000000,0.000000,1.030850e+05
50%,0.000000,2190.359362,43.000000,1.000000,0.000000,0.000000,1.450208e+05
75%,1.000000,3577.539706,53.000000,1.000000,1.000000,0.000000,2.033455e+05
max,5.000000,15267.541183,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Больше аномальных значений не выявлено. Можно перенумеровать индексы строк и перейти к заполенению пропусков.

In [15]:
data = data.reset_index(drop=True)
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].median())
data['total_income'] = data['total_income'].fillna(data['total_income'].median())
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [16]:
data['gender'].value_counts()

F      14118
M       7230
XNA        1
Name: gender, dtype: int64

В столбце `gender` присутсвутет одно значение "XNA". Удалим строку с этим значением.

In [17]:
data = data[data['gender'] != 'XNA']
data['gender'].value_counts()

F    14118
M     7230
Name: gender, dtype: int64

### Шаг 2.3. Изменение типов данных.

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21348 entries, 0 to 21348
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21348 non-null  int64  
 1   days_employed     21348 non-null  float64
 2   dob_years         21348 non-null  int64  
 3   education         21348 non-null  object 
 4   education_id      21348 non-null  int64  
 5   family_status     21348 non-null  object 
 6   family_status_id  21348 non-null  int64  
 7   gender            21348 non-null  object 
 8   income_type       21348 non-null  object 
 9   debt              21348 non-null  int64  
 10  total_income      21348 non-null  float64
 11  purpose           21348 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


In [19]:
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,3280.806279,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


В столбцах `total_income` и  `days_employed` представлены значения с шестью знаками после запятой. Такая точность излишняя. Заменим значения на целочисленные.

In [20]:
data['days_employed'] = data['days_employed'].astype('int64')
data['total_income'] = data['total_income'].astype('int64')
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,3280,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


### Шаг 2.4. Удаление дубликатов.

Для начала найдём явные дубликаты в данных.

In [21]:
data.duplicated().sum()

54

In [22]:
data['children'].value_counts()

0    14079
1     4849
2     2042
3      328
4       41
5        9
Name: children, dtype: int64

In [23]:
data['education'].value_counts()

среднее                13640
высшее                  4674
СРЕДНЕЕ                  768
Среднее                  702
неоконченное высшее      663
ВЫСШЕЕ                   271
Высшее                   266
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

In [24]:
data['education_id'].value_counts()

1    15110
0     5211
2      739
3      282
4        6
Name: education_id, dtype: int64

In [25]:
data['family_status'].value_counts()

женат / замужем          12283
гражданский брак          4143
Не женат / не замужем     2788
в разводе                 1183
вдовец / вдова             951
Name: family_status, dtype: int64

In [26]:
data['gender'].value_counts()

F    14118
M     7230
Name: gender, dtype: int64

In [27]:
data['debt'].value_counts()

0    19623
1     1725
Name: debt, dtype: int64

In [28]:
data['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     769
сыграть свадьбу                           765
операции с недвижимостью                  672
покупка коммерческой недвижимости         658
покупка жилья для сдачи                   650
операции с коммерческой недвижимостью     645
операции с жильем                         643
покупка жилья для семьи                   639
покупка жилья                             638
жилье                                     637
операции со своей недвижимостью           629
недвижимость                              629
строительство собственной недвижимости    627
строительство жилой недвижимости          622
строительство недвижимости                620
покупка своего жилья                      619
покупка недвижимости                      618
ремонт жилью                              608
покупка жилой недвижимости                602
на покупку своего автомобиля              502
заняться высшим образованием      

Из представленной выше информации видно, что в столбцах `education` и `purpose` присутствуют неявные дубликаты. Они связаны с человеческим фактором.

В столбце `education` можно устранить проблему с помощью приведения значений к нижнему регистру.

В столбце `purpose` можно устранить проблему с помощью категоризации данных, что будет предпринято в следующих этапах проекта.

In [29]:
data['education'] = data['education'].str.lower()
data['education'].value_counts()

среднее                15110
высшее                  5211
неоконченное высшее      739
начальное                282
ученая степень             6
Name: education, dtype: int64

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [30]:
# Датафрейм-словарь с данными об образовании
education_dict = data[['education_id','education']]
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
education_dict

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [31]:
# Датафрейм-словарь с данными о семейном статусе
family_status_dict = data[['family_status_id','family_status']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
family_status_dict

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


In [32]:
family_status_dict['family_status'] = family_status_dict['family_status'].str.lower()
family_status_dict

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


In [33]:
# Удаление столбцов в исходном датафрейме
data = data.drop(columns = ['education', 'family_status'])
data.head(5)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,3280,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


### Шаг 2.6. Категоризация дохода.

Проведем категоризацию клиентов в зависимости от доходов:
* 0–30000 — `E`;
* 30001–50000 — `D`;
* 50001–200000 — `C`;
* 200001–1000000 — `B`;
* 1000001 и выше — `A`.

In [34]:
def income_category(income):
    if 0 <= income <= 30000:
        return 'E'
    elif 30001 <= income <= 50000:
        return 'D'
    elif 50001 <= income <= 200000:
        return 'C'
    elif 200001 <= income <= 1000000:
        return 'B'
    else:
        return 'A'

In [35]:
data['total_income_category'] = data['total_income'].apply(income_category)
data.head(5)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,3280,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


### Шаг 2.7. Категоризация целей кредита.

In [36]:
print(data['purpose'].unique())

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


In [37]:
def purpose_category(purpose):
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    elif 'недвижимост' in purpose or 'жиль' in purpose:
        return 'операции с недвижимостью'
    elif 'образован' in purpose:
        return 'получение образования'
    elif 'автомобил' in purpose:
        return 'операции с автомобилем'
    else:
        return f'категория "{purpose}" не определена'

In [38]:
data['purpose_category'] = data['purpose'].apply(purpose_category)
data['purpose_category'].value_counts()

операции с недвижимостью    10756
операции с автомобилем       4278
получение образования        3989
проведение свадьбы           2325
Name: purpose_category, dtype: int64

### Ответы на вопросы.

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [39]:
def my_mean(x): return '{:.2%} '.format(x.mean())

data.groupby('children')['debt'].agg(['count', 'sum', my_mean])

,count,sum,my_mean
children,,,
0,14079,1058,7.51%
1,4849,442,9.12%
2,2042,194,9.50%
3,328,27,8.23%
4,41,4,9.76%
5,9,0,0.00%


##### Вывод 1:

В среднем чем больше детей, тем больше вероятность того, что клиент не вернёт кредит в срок. Однако количество клиентов с более чем 2 детьми достаточно мало, поэтому данная зависимость нуждается в дальнейшем уточнении. 

##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [40]:
display(family_status_dict)

def my_mean(x): return '{:.2%} '.format(x.mean())

a = data.groupby('family_status_id')['debt'].agg(['count', 'sum', my_mean])
display(a)

a.reset_index().replace({'family_status_id': family_status_dict.family_status.to_dict()})

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


,count,sum,my_mean
family_status_id,,,
0,12283,924,7.52%
1,4143,383,9.24%
2,951,62,6.52%
3,1183,84,7.10%
4,2788,272,9.76%


,family_status_id,count,sum,my_mean
0,женат / замужем,12283,924,7.52%
1,гражданский брак,4143,383,9.24%
2,вдовец / вдова,951,62,6.52%
3,в разводе,1183,84,7.10%
4,не женат / не замужем,2788,272,9.76%


##### Вывод 2:

В среднем вероятность того, что люди в гражданском браке и не женатые / не замужем не вернут кредит в срок, больше, чем у людей в разводе и женатых / замежем. Вдовцы / вдовы в наименьшей степени склонны иметь задолженность по кредиту.

##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [41]:
data.pivot_table(index = 'total_income_category', values = 'debt', aggfunc = ['mean', 'count'])

,mean,count
,debt,debt
total_income_category,,
A,0.080000,25
B,0.070685,4994
C,0.084404,15959
D,0.060345,348
E,0.090909,22


##### Вывод 3:

Люди, получающие доход от 0 до 30000, в наибольшей степени склонны не возвращать кредит в срок. Чуть меньше склонны не возвращать кредит в срок люди с доходом от 50001 до 200000 и выше 1000001. Меньше всего задолженностей возникает у лиц с доходом от 30001 до 50000.

Однако стоит отметить тот факт, что наблюдяется явный перекос выборки в сторону людей с доходом 200001–1000000 (B) и 50001–200000 (С), поэтому данные выводы нуждаются в дальнейшем подтверждении.

##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

In [42]:
data.pivot_table(index = 'purpose_category', values = 'debt', aggfunc = ['mean', 'count'])

,mean,count
,debt,debt
purpose_category,,
операции с автомобилем,0.093034,4278
операции с недвижимостью,0.072239,10756
получение образования,0.092504,3989
проведение свадьбы,0.077849,2325


##### Вывод 4: 

В среднем люди, получившие кредит для операций с недвижимостью или проведения свадьбы, возвращают кредит в срок чаще, чем люди, взявшие кредит для операций с автомобилем или получения образования.

## Общий вывод:

Как показало исследование, существуют следующие факторы, снижающие вероятность просрочки кредита:
* клиент не имеет детей;
* клиент - вдовец / вдова;
* клиент имеет доход от 30001 до 50000;
* клиент берёт кредит для проведения свадьбы или операций с недвижимостью.

Основываясь на предоставленной выборке, можно утверждать, что:
* с точки зрения <b>количества детей</b> самыми надежными являются клиенты <b>без детей</b> (7.54% просроченных кредитов), а наименее надежны клиенты <b>с 4 детьми</b> (9.76% просроченных кредитов);
* с точки зрения <b>семейного положения</b> самыми надежными являются клиенты <b>вдовцы и вдовы</b> (7.54% просроченных кредитов), а наименее надежны <b>неженатые и незамужние </b>клиенты (9.77% просроченных кредитов);
* с точки зрения <b>уровня дохода</b> самыми надежными являются клиенты <b>с доходом от 30 001 до 50 000</b>  (6.03% просроченных кредитов), а наименее надежны клиенты <b>с доходом от 0 до 30 000</b> (9.09% просроченных кредитов);
* с точки зрения <b>целей кредита</b> самыми надежными являются клиенты, получающие деньги <b>для операций с недвижимостью</b> (7.24% просроченных кредитов), а наименее надежные клиенты, получающие деньги <b>для операций с автомобилем</b> (9.31% просроченных кредитов).